<a href="https://colab.research.google.com/github/wrcarpenter/MMA-Handicapping-Model/blob/main/Code/UFC_Bout_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing

In [13]:
#Importing
import requests
import csv 
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
from google.colab import files
import datetime
from datetime import date
from pytz import timezone
eastern = timezone('US/Eastern')
import threading 
from concurrent.futures import ThreadPoolExecutor
import multiprocessing
cores = multiprocessing.cpu_count()

# Read in Dataset


In [14]:
source = 'https://raw.githubusercontent.com/wrcarpenter/MMA-Handicapping-Model/main/Data/ufcBouts_v3.csv'  
df      = pd.read_csv(source, header=0) 
df_orig = df # preserve a copy in case

# Data Summary 

Review data composition for general understanding and debugging purposes. The current construction of the raw dataset should contain 29 columns. 

In [ ]:
# Explore the dataset 
print(df.columns)
# Columns in dataset 
print(df.shape[1]) # raw dataset should have 29 columns 
# Unique fighters
print("Number of Unique Fighters: ", len(pd.unique(df['name'])))  # unique fighters
# Number of fights (nows in dataset)
print("Total Fights Recorded: ", len(df))
# Number of unique fights in the dataset 
print("Number of Unique Fights: ", len(pd.unique(df['fight_link'])))  


# Create Variables 

First need to decide what fights in the dataset are upcoming or not. Probably the best way to approach fights is only keep data on completed (all stats are there) type of fights because its unclear to me how the newer ones update.

Need to handle missing data: birthdays, reach, height, etc.

Clean data for various variables and then being calculations to generate for model. 

In [ ]:
# Eliminate any missing DOB (can fill this in later)
df = df[df['dob'] != "--"]  # drops missing birthdays from the data 
print(len(df) - len(df_orig))  # drops 865 obs
# Sort dob
df['dob'] = df['dob'].replace(',', '', regex=True)  # replace commas
df['dob'] = df['dob'].replace('--', '', regex=True)  # replace commas
df['dob'] = pd.to_datetime(df['dob'], format="%m/%d/%Y")
# Sort event date
df['event_date'] = df['event_date'].replace(',', '', regex=True)  # replace commas 
df['event_date'] = pd.to_datetime(df['event_date'], format="%m/%d/%Y")
# Sorts in new data 
df = df.sort_values(by=['fighter_profile', 'event_date'], ascending=True)
# Show results
df.loc[df['name'] == 'Jon Jones', ['event_date', 'event', 'total_fights']]

In [ ]:
# Current age of fighter at date time (years)
df['current_age'] = (df['event_date'] - df['dob']) / np.timedelta64(1, 'Y')

# Number of previous recorded fights in data 
df['ones'] = 1
df['prev_num_fights'] = df.groupby(['fighter_profile'])['ones'].cumsum() - 1
# df.drop(columns=['total_fights'])

# Weeks since previous fight
df['wks_since_last_fight'] = df.groupby(['fighter_profile'])['event_date'].diff()  # total time between date values
df['wks_since_last_fight'] = df['wks_since_last_fight'] / np.timedelta64(1, 'W')   # this includes some NaN 
 
# Result of last fight 
# Need to breakdown the results by some kind of number pattern 

# Result of second to last fight  

# KOed in last four fights?    true or false
# Subed in last four fights?   true or false

# Testing results 
df.loc[df['name'] == 'Jon Jones', ['event_date', 'event', 'prev_num_fights']]
df.loc[df['name'] == 'Conor McGregor', ['event_date', 'event', 'prev_num_fights', 'wks_since_last_fight']]


In [ ]:
df['fighter_result'].value_counts()  # win, loss, no-contest, draw

df['method'].value_counts()

df.columns